* Master DAC - BDLE
* Author: Mohamed-Amine Baazizi
* Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
* Email: mohamed-amine.baazizi@lip6.fr
* October 2023

## Outline

Documentation

* Delta https://docs.delta.io/0.4.0/delta-intro.html
* API Reference https://docs.delta.io/0.4.0/api/python/index.html

Organization
* Demo1 ->3 and additional material: illustrate concepts of the lecture
* Use cases: the exercice(s) to solve








check compatibility https://docs.delta.io/latest/releases.html

## Prerequisite

In [ ]:
!pip install --upgrade -q pyspark==3.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade -q delta-spark==2.4

In [ ]:
!pip list|grep delta

delta-spark                      2.4.0


In [ ]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

local = "local[*]"
appName = "DeltaLake"
localConfig = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").\
  set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog").\
  set("spark.jars.packages","io.delta:delta-core_2.12:2.4.0").\
  set("spark.databricks.delta.schema.autoMerge.enabled","true")


spark = SparkSession.builder.config(conf = localConfig).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [ ]:
spark

### imports

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *


## Demo1

### load the data into delta

In [ ]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



### update the data
#### overwrite

In [ ]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
|  5|
|  6|
+---+



#### conditional overwrite

In [ ]:
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })


In [ ]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|108|
|  9|
|  5|
|106|
+---+



In [ ]:
# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+



In [ ]:
# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



### view history

In [ ]:
history = deltaTable.history()
history.printSchema()
history.show()

root
 |-- version: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- userId: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- operationParameters: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- job: struct (nullable = true)
 |    |-- jobId: string (nullable = true)
 |    |-- jobName: string (nullable = true)
 |    |-- runId: string (nullable = true)
 |    |-- jobOwnerId: string (nullable = true)
 |    |-- triggerType: string (nullable = true)
 |-- notebook: struct (nullable = true)
 |    |-- notebookId: string (nullable = true)
 |-- clusterId: string (nullable = true)
 |-- readVersion: long (nullable = true)
 |-- isolationLevel: string (nullable = true)
 |-- isBlindAppend: boolean (nullable = true)
 |-- operationMetrics: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- userMetadata: string (nullable =

In [ ]:
history.select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
!ls -al /tmp/delta-table/_delta_log

total 48
drwxr-xr-x 2 root root 4096 Nov  2 22:07 .
drwxr-xr-x 3 root root 4096 Nov  2 22:07 ..
-rw-r--r-- 1 root root 1259 Nov  2 22:02 00000000000000000000.json
-rw-r--r-- 1 root root   20 Nov  2 22:02 .00000000000000000000.json.crc
-rw-r--r-- 1 root root 1324 Nov  2 22:03 00000000000000000001.json
-rw-r--r-- 1 root root   20 Nov  2 22:03 .00000000000000000001.json.crc
-rw-r--r-- 1 root root 1496 Nov  2 22:04 00000000000000000002.json
-rw-r--r-- 1 root root   20 Nov  2 22:04 .00000000000000000002.json.crc
-rw-r--r-- 1 root root 1492 Nov  2 22:05 00000000000000000003.json
-rw-r--r-- 1 root root   20 Nov  2 22:05 .00000000000000000003.json.crc
-rw-r--r-- 1 root root 1661 Nov  2 22:07 00000000000000000004.json
-rw-r--r-- 1 root root   24 Nov  2 22:07 .00000000000000000004.json.crc


In [ ]:
!cat /tmp/delta-table/_delta_log/00000000000000000000.json

{"commitInfo":{"timestamp":1698962579073,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"2","numOutputRows":"5","numOutputBytes":"977"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"c320343e-f16d-41f9-a4a1-ac293e234240"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"fe4f6dfd-8037-4588-93d3-4b1325eb3109","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1698962575745}}
{"add":{"path":"part-00000-c2455c7c-fc67-46b2-963c-71a0d43bbb50-c000.snappy.parquet","partitionValues":{},"size":486,"modificationTime":1698962578647,"dataChange":true,"stats":"{\"numRecords\":2,\"minValues\":{\"id\":0},\"maxValues\":{\"id\":1},\"nullCount\":{\"id\":0}}"}}
{"add":{"path":

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [ ]:
df = spark.read.format("delta").option("versionAsOf", 1).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
|  5|
|  6|
+---+



## Datagen


### Persons

In [ ]:
# create sample data
data = [("12345", "Alice", 25, "123 Main St"),
        ("67890", "Bob", 30, "456 Oak Ave"),
        ("24680", "Charlie", 35, "789 Elm St")]

# create a DataFrame from the sample data
df = spark.createDataFrame(data, ["serial", "name", "age", "address"])

# write the DataFrame to Delta format
df.write.format("delta").save("/tmp/persons")

In [ ]:
newdata = [("78120", "Dan", 42, "432 Holly Rd"),
        ("97362", "Lorry", 40, "290 Wise Ave")]


# create a DataFrame
newPersons = spark.createDataFrame(newdata, ["serial", "name", "age", "address"])

### Salaries

In [ ]:
salaries = [("12345", 45000),
        ("67890", 52000),
        ("24680", 36000),
        ("78120", 60000),
        ("97362",38000)]

# create a DataFrame from the sample data
df = spark.createDataFrame(salaries, ["serial", "salary"])

# write the DataFrame to Delta format
df.write.format("delta").save("/tmp/salaries")

In [ ]:
new_salaries = [("12345", 47000),
        ("67890", 50000),
        ("24680", 46000),
        ("78120", 61000),
        ("97362",39000)]

# create a DataFrame
newSalaries = spark.createDataFrame(new_salaries, ["serial", "salary"])

### Sales

In [ ]:
sales = [("CHA_2",2,60),("BED_4",1,300),("SHO_15",2,60)]

# create a DataFrame from the sample data
df = spark.createDataFrame(sales, ["product_id", "quantity", "totalprice"])
df.write.format("delta").save("/tmp/sales")

In [ ]:
new_sales = [("SHO_15",3,90),("CHA_2",1,30),("BED_6",1,200)]

newSales = spark.createDataFrame(new_sales, ["product_id", "quantity", "totalprice"])

### Products

In [ ]:
products_list = [("CHA_2","Furniture","blue"),("BED_4","Furniture","brown"),("SHO_15","Cloth","black")]
products = spark.createDataFrame(products_list, ["product_id", "category", "color"])

## Demo2

### Q1. Adding new tuples
Consider the Delta table `person` with the following columns: serial, name, age, and address. You have a new dataset `newPersons` with the same columns, but with additional records. Write a merge statement to update the Delta table with the new records.


In [ ]:

#load the persons table
delta_persons = DeltaTable.forPath(spark, "/tmp/persons")

# Define the merge condition
merge_condition = "target.serial = source.serial"

# Define the merge statement
delta_persons.alias("target").merge(
    newPersons.alias("source"), merge_condition
).whenNotMatchedInsertAll().execute()



#### verification

In [ ]:
delta_persons.toDF().show()

+------+-------+---+------------+
|serial|   name|age|     address|
+------+-------+---+------------+
| 97362|  Lorry| 40|290 Wise Ave|
| 12345|  Alice| 25| 123 Main St|
| 78120|    Dan| 42|432 Holly Rd|
| 67890|    Bob| 30| 456 Oak Ave|
| 24680|Charlie| 35|  789 Elm St|
+------+-------+---+------------+



In [ ]:
delta_persons.history().select("version","operation","operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationParameters                                                                                                                                           |
+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |MERGE    |{predicate -> ["(serial#7211 = serial#6060)"], matchedPredicates -> [], notMatchedPredicates -> [{"actionType":"insert"}], notMatchedBySourcePredicates -> []}|
|0      |WRITE    |{mode -> ErrorIfExists, partitionBy -> []}                                                                                                                    |
+-------+---------+--------------------------------------------------------------------------------------

### Q2: updating existing tuples
Assume you have a Delta table `salaries` with columns serial and salary. You want to update the salary of the employees who earn less than 50,000. You have a new dataset, `newSalaries` with the same columns but with updated salary information. Write a merge statement to update the `salaries` table with the new salary information.


In [ ]:
#load the persons table
delta_salaries = DeltaTable.forPath(spark, "/tmp/salaries")

# Define
merge_condition = "target.serial = source.serial and target.salary<50000"
update_expression = { "salary": "source.salary" }

# merge statement
delta_salaries.alias("target") \
  .merge(
    newSalaries.alias("source"), merge_condition ) \
  .whenMatchedUpdate(set = update_expression) \
  .execute()


#### verification

In [ ]:
delta_salaries.toDF().show()


+------+------+
|serial|salary|
+------+------+
| 78120| 60000|
| 97362| 39000|
| 24680| 46000|
| 12345| 47000|
| 67890| 52000|
+------+------+



In [ ]:
delta_persons.history().select("version","operation","operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationParameters                                                                                                                                           |
+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |MERGE    |{predicate -> ["(serial#7211 = serial#6060)"], matchedPredicates -> [], notMatchedPredicates -> [{"actionType":"insert"}], notMatchedBySourcePredicates -> []}|
|0      |WRITE    |{mode -> ErrorIfExists, partitionBy -> []}                                                                                                                    |
+-------+---------+--------------------------------------------------------------------------------------

### Q3: adding new tuples and updating existing ones
You have a Delta table `sales` with columns `product_id`, `quantity`, and `totalprice`. Write a merge statement to insert the new products from a dataframe `newSales` into `sales` and to make sure that, for existing products `sales` has the sum of the quantity and totalprice.


In [ ]:
#load the sales table
delta_sales = DeltaTable.forPath(spark, "/tmp/sales")

In [ ]:
delta_sales.toDF().show()

+----------+--------+----------+
|product_id|quantity|totalprice|
+----------+--------+----------+
|     BED_4|       1|       300|
|    SHO_15|       2|        60|
|     CHA_2|       2|        60|
+----------+--------+----------+



In [ ]:
newSales.show()

+----------+--------+----------+
|product_id|quantity|totalprice|
+----------+--------+----------+
|    SHO_15|       3|        90|
|     CHA_2|       1|        30|
|     BED_6|       1|       200|
+----------+--------+----------+



In [ ]:
# Define
merge_condition = "target.product_id = source.product_id"
update_expression = { "quantity": "target.quantity+source.quantity",  "totalprice": "target.totalprice+source.totalprice"}

# merge statement
delta_sales.alias("target") \
  .merge(
    newSales.alias("source"), merge_condition ) \
  .whenMatchedUpdate(set = update_expression) \
  .whenNotMatchedInsertAll()\
  .execute()

#### verification

In [ ]:
delta_sales.toDF().show()

+----------+--------+----------+
|product_id|quantity|totalprice|
+----------+--------+----------+
|     BED_4|       1|       300|
|     BED_6|       1|       200|
|     CHA_2|       3|        90|
|    SHO_15|       5|       150|
+----------+--------+----------+



In [ ]:
delta_persons.history().select("version","operation","operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationParameters                                                                                                                                           |
+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |MERGE    |{predicate -> ["(serial#7211 = serial#6060)"], matchedPredicates -> [], notMatchedPredicates -> [{"actionType":"insert"}], notMatchedBySourcePredicates -> []}|
|0      |WRITE    |{mode -> ErrorIfExists, partitionBy -> []}                                                                                                                    |
+-------+---------+--------------------------------------------------------------------------------------

### Q4: Merge tables with different schemas
You have a Delta table `sales` with a column `product_id`, among other.  Write a merge statement to update `sales` with  information about products using a dataset `productInfo` which contains the columns `product_id`, `category` and `color`, when available.

In [ ]:
products.createOrReplaceTempView("products")

spark.sql("""MERGE INTO delta.`/tmp/sales` t
USING products s
ON t.product_id = s.product_id
WHEN MATCHED
  THEN UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [ ]:
spark.sql("""select * from  delta.`/tmp/sales` """).show()

+----------+--------+----------+---------+-----+
|product_id|quantity|totalprice| category|color|
+----------+--------+----------+---------+-----+
|     BED_4|       1|       300|Furniture|brown|
|     BED_6|       1|       200|     null| null|
|     CHA_2|       3|        90|Furniture| blue|
|    SHO_15|       5|       150|    Cloth|black|
+----------+--------+----------+---------+-----+



### Q4: Merge tables with different schemas (Méthode 2)
En ajoutant d'abord les colonnes manquantes avec des valeurs nulles. Ainsi se ramener au cas où toutes les colonnes sont présentes dans la target.

In [ ]:
# Test avec le système de requete spark

sales_2 = [("CHA_2",2,60),("BED_4",1,300),("SHO_15",2,60)]

# create a DataFrame from the sample data
df_2 = spark.createDataFrame(sales_2, ["product_id", "quantity", "totalprice"])
df_2.write.format("delta").mode("overwrite").save("/tmp/sales_2")

In [ ]:
new_sales_2 = [("SHO_15",3,90),("CHA_2",1,30),("BED_6",1,200)]

newSales_2 = spark.createDataFrame(new_sales_2, ["product_id", "quantity", "totalprice"])

In [ ]:
products_list_2 = [("CHA_2","Furniture","blue"),("BED_4","Furniture","brown"),("SHO_15","Cloth","black")]
products_2 = spark.createDataFrame(products_list_2, ["product_id", "category", "color"])

In [ ]:
delta_sales_2 = DeltaTable.forPath(spark, "/tmp/sales_2")

# ajout des colonnes category et color (avec des valeurs nulles par défaut)
delta_sales_2.toDF().withColumn("category", lit(None)).withColumn("color", lit(None))

# Define
merge_condition = "target.product_id = source.product_id"
update_expression = {
    "category": "source.category",
    "color": "source.color"
}

# merge statement
delta_sales_2.alias("target") \
  .merge(
    products_2.alias("source"), merge_condition ) \
  .whenMatchedUpdate(set = update_expression) \
  .execute()

In [ ]:
delta_sales_2.toDF().show()

+----------+--------+----------+---------+-----+
|product_id|quantity|totalprice| category|color|
+----------+--------+----------+---------+-----+
|     BED_4|       1|       300|Furniture|brown|
|    SHO_15|       2|        60|    Cloth|black|
|     CHA_2|       2|        60|Furniture| blue|
+----------+--------+----------+---------+-----+



#### verification

In [ ]:
delta_persons.history().select("version","operation","operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationParameters                                                                                                                                           |
+-------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |MERGE    |{predicate -> ["(serial#7211 = serial#6060)"], matchedPredicates -> [], notMatchedPredicates -> [{"actionType":"insert"}], notMatchedBySourcePredicates -> []}|
|0      |WRITE    |{mode -> ErrorIfExists, partitionBy -> []}                                                                                                                    |
+-------+---------+--------------------------------------------------------------------------------------

## Demo 3: Attaching constraints

### Not-null constraint

In [ ]:
spark.sql("""
CREATE TABLE default.persons (
    serial INT NOT NULL,
    name STRING,
    birthDate TIMESTAMP,
    address STRING
  ) USING DELTA;

""")

DataFrame[]

In [ ]:
spark.sql(""" DESCRIBE DETAIL default.persons """).show()

+------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|                name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|47c4a377-0232-469...|spark_catalog.def...|       null|file:/content/spa...|2023-11-03 00:00:...|2023-11-03 00:00:...|              []|       0|          0|        {}|               1|               2|[appendOnly, inva...|
+------+--------------------+--------------------+-----------+------------------

In [ ]:
!ls /content/spark-warehouse/persons

_delta_log


In [ ]:
spark.sql("""select * from default.persons """).show()


+------+----+---------+-------+
|serial|name|birthDate|address|
+------+----+---------+-------+
+------+----+---------+-------+



In [ ]:
spark.sql("""insert into default.persons values (12345, "Alice","2000-02-01" ,"123 Main St") """)

DataFrame[]

In [ ]:
spark.sql("""select * from default.persons """).show()


+------+-----+-------------------+-----------+
|serial| name|          birthDate|    address|
+------+-----+-------------------+-----------+
| 12345|Alice|2000-02-01 00:00:00|123 Main St|
+------+-----+-------------------+-----------+



Can we run the following statement?

In [ ]:
# spark.sql("""insert into default.persons values (null, "Bob","1996-03-14" ,"456 Oak Ave") """)

Non parceque, violation de la contrainte sur serial qui doit être non null.

### Predicate constraint

In [ ]:
spark.sql(""" ALTER TABLE default.persons ADD CONSTRAINT birthdate CHECK (birthDate > '2000-01-01'); """)

DataFrame[]

In [ ]:
spark.sql("""SHOW TBLPROPERTIES default.persons""").show(truncate=False)

+---------------------------+------------------------+
|key                        |value                   |
+---------------------------+------------------------+
|delta.constraints.birthdate|birthDate > '2000-01-01'|
|delta.minReaderVersion     |1                       |
|delta.minWriterVersion     |3                       |
+---------------------------+------------------------+



Can we run the following statement?

In [ ]:
spark.sql("""insert into default.persons values (47962, "Bob","2003-03-14" ,"456 Oak Ave") """)

DataFrame[]

In [ ]:
# spark.sql("""insert into default.persons values (47962, "Bob","1999-03-14" ,"456 Oak Ave") """)

Non, car bithdate check non satisfaite. bithdate <= 2000-01-01.

## Use case 1

### Data import

In [ ]:
! wget https://nuage.lip6.fr/s/BbQ9rzGHKJexKYp/download/sales.tar -O /tmp/sales.tar

--2023-11-03 00:23:45--  https://nuage.lip6.fr/s/BbQ9rzGHKJexKYp/download/sales.tar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548352 (536K) [application/x-tar]
Saving to: ‘/tmp/sales.tar’

/tmp/sales.tar      100%[===================>] 535.50K   798KB/s    in 0.7s    

2023-11-03 00:23:46 (798 KB/s) - ‘/tmp/sales.tar’ saved [548352/548352]



In [ ]:
!mkdir /tmp/delta

In [ ]:
! tar xvf /tmp/sales.tar -C /tmp/delta

sales/
sales/._salesOriginal.csv
sales/salesOriginal.csv
sales/march23_sales.csv


In [ ]:
!ls /tmp/delta/sales

march23_sales.csv  salesOriginal.csv


In [ ]:
# !rm -rf /tmp/delta/deltaSales

In [ ]:
# original
orginal_sales = spark.read.csv("/tmp/delta/sales/salesOriginal.csv", header=True, inferSchema=True)
orginal_sales.write.format("delta").partitionBy("category").save("/tmp/delta/deltaSales")

In [ ]:
print("count: %d \n schema: " % orginal_sales.count())
orginal_sales.dtypes

count: 4916 
 schema: 


[('saleid', 'string'),
 ('saledate', 'date'),
 ('quantity', 'double'),
 ('unitprice', 'double'),
 ('shopid', 'string'),
 ('city', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('shopsize', 'string'),
 ('productid', 'string'),
 ('category', 'string'),
 ('subcategory', 'string'),
 ('size', 'string'),
 ('purchaseprice', 'double'),
 ('color', 'string'),
 ('brand', 'string')]

In [ ]:
# march 2023 sales
march23_sales = spark.read.csv("/tmp/delta/sales/march23_sales.csv", header=True, inferSchema=True)
print("count: %d \n schema: " % march23_sales.count())
march23_sales.dtypes

count: 84 
 schema: 


[('saleid', 'string'),
 ('saledate', 'date'),
 ('quantity', 'double'),
 ('unitprice', 'double'),
 ('shopid', 'string'),
 ('city', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('shopsize', 'string'),
 ('productid', 'string'),
 ('category', 'string'),
 ('subcategory', 'string'),
 ('size', 'string'),
 ('purchaseprice', 'double'),
 ('color', 'string'),
 ('brand', 'string')]

### Load the delta table

In [ ]:
deltaSales = DeltaTable.forPath(spark, "/tmp/delta/deltaSales")

In [ ]:
deltaSales.detail().show()

+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|7388a244-50fb-491...|null|       null|file:/tmp/delta/d...|2023-11-03 01:55:...|2023-11-03 01:55:...|      [category]|       2|      50284|        {}|               1|               2|[appendOnly, inva...|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+---------

In [ ]:
deltaSales.history().select("version","operation","operationParameters").show(truncate=False)

+-------+---------+----------------------------------------------------+
|version|operation|operationParameters                                 |
+-------+---------+----------------------------------------------------+
|0      |WRITE    |{mode -> ErrorIfExists, partitionBy -> ["category"]}|
+-------+---------+----------------------------------------------------+



### Adding new tuples
Write a merge statement to include the march 2023 records into `deltaSales`

In [ ]:
# merge

# Define the merge condition
merge_condition = "target.saleid = source.saleid"

# Define the merge statement
deltaSales.alias("target") \
  .merge(
      march23_sales.alias("source"),
      merge_condition) \
  .whenNotMatchedInsertAll() \
  .execute()


In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
deltaSales.toDF().count()

5000

### Updating tuples
Write an update statement that increases the prices of products sold on 2023, based on their category, as follows: furniture -> 05%, others -> 10%

In [ ]:
deltaSales.toDF().where("saledate >= '2023-01-01' and category='Furniture'").count()

882

In [ ]:
# update

deltaSales.update(
    condition = expr(" saledate >= '2023-01-01'"),
    set={ "unitprice": when(expr("category == 'furniture'"), expr("unitprice * 1.05"))\
                              .otherwise(expr("unitprice * 1.10"))
        }
)


In [ ]:
deltaSales.history().select("version","operation","operationParameters").show()

+-------+---------+--------------------+
|version|operation| operationParameters|
+-------+---------+--------------------+
|      2|   UPDATE|{predicate -> ["(...|
|      1|    MERGE|{predicate -> ["(...|
|      0|    WRITE|{mode -> ErrorIfE...|
+-------+---------+--------------------+



In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
deltaSales.toDF().where("saledate >= '2023-01-01' and category!='Furniture'").count()

1680

In [ ]:
deltaSales.toDF().where("saledate < '2023-01-01'").count()

2438

In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                            

### Removing old records
remove all sales older than 01-Jan-2023. How many records remain?

In [ ]:
deltaSales.toDF().where("saledate < '2023-01-01'").count()

2438

In [ ]:
# delete

deltaSales.delete(condition = expr("saledate < '2023-01-01'"))


In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
deltaSales.toDF().count()

2562

### History viewing
Show the records that have been deleted. Use the metadata information and use dataframe operators.  

In [ ]:
deltaSales.toDF().where("saledate < '2023-01-01'").count()

0

In [ ]:
spark.read.format("delta").option("versionAsOf", 3).load("/tmp/delta/deltaSales").where("saledate < '2023-01-01'").count()


0

In [ ]:
df_del = spark.read.format("delta").option("versionAsOf", 2).load("/tmp/delta/deltaSales").where("saledate < '2023-01-01'")
df_del.show()


+----------+----------+--------+---------+-------+-------------+----------+-------+--------+---------+--------+-----------+----+-------------+-----+--------+
|    saleid|  saledate|quantity|unitprice| shopid|         city|     state|country|shopsize|productid|category|subcategory|size|purchaseprice|color|   brand|
+----------+----------+--------+---------+-------+-------------+----------+-------+--------+---------+--------+-----------+----+-------------+-----+--------+
|S000000181|2022-12-31|     1.0|     60.0|shop_21|      Houston|     Texas|    USA|  medium|   SHO_59|   Cloth|      Shoes|   9|         44.0|brown|NewBrand|
|S000000182|2022-12-30|     1.0|     60.0|shop_22|  San Antonio|     Texas|    USA|   small|   SHO_60|   Cloth|      Shoes|  10|         46.0|black|   Basic|
|S000000190|2022-12-22|     1.0|     20.0|shop_30|        Milan|  Lombardy|  Italy|  medium|    TSH_7|   Cloth|     Tshirt|  XS|         17.0| blue|NewBrand|
|S000000191|2022-12-21|     1.0|     20.0|shop_31|  

In [ ]:
df_del.count()


2438

### Vacuuming old records
Permanently remove the deleted records using `vacuum`. Check the history again and make sure that the removal has been performed.

In [ ]:
deltaSales.vacuum()

DataFrame[]

In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

In [ ]:
! ls -rhl /tmp/delta/deltaSales

total 12K
drwxr-xr-x 2 root root 4.0K Nov  3 02:18  _delta_log
drwxr-xr-x 2 root root 4.0K Nov  3 02:15 'category=Furniture'
drwxr-xr-x 2 root root 4.0K Nov  3 02:15 'category=Cloth'


In [ ]:
deltaSales.toDF().count()

2562

count the rows at version 0, 3 and 5 and analyse

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta/deltaSales")
df.count()

4916

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 3).load("/tmp/delta/deltaSales")
df.count()

2562

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 5).load("/tmp/delta/deltaSales")
df.count()

2562

In [ ]:
deltaSales.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation   |operationMetrics                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Additional material

### Generated columns
Consider the sales data from the use case, create a delta table called `deltaSalesDate` with three additional columns `year`, `month` and `day` derived from the `saledate` column of the original data.

In [ ]:
orginal_sales = spark.read.csv("/tmp/delta/sales/salesOriginal.csv", header=True, inferSchema=True)
orginal_sales.printSchema()

root
 |-- saleid: string (nullable = true)
 |-- saledate: date (nullable = true)
 |-- quantity: double (nullable = true)
 |-- unitprice: double (nullable = true)
 |-- shopid: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shopsize: string (nullable = true)
 |-- productid: string (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- size: string (nullable = true)
 |-- purchaseprice: double (nullable = true)
 |-- color: string (nullable = true)
 |-- brand: string (nullable = true)



#### option1: augment `orginal_sales` with the three columns

In [ ]:
from pyspark.sql.functions import *

In [ ]:
sales_with_date_components = orginal_sales.withColumn('year',year(col("saledate")))\
.withColumn('month',month(col("saledate")))\
.withColumn('day',dayofmonth(col("saledate")))
sales_with_date_components.show(10)

+----------+----------+--------+---------+-------+-------------+----------+-------+--------+---------+---------+-----------+------+-------------+-----+--------+----+-----+---+
|    saleid|  saledate|quantity|unitprice| shopid|         city|     state|country|shopsize|productid| category|subcategory|  size|purchaseprice|color|   brand|year|month|day|
+----------+----------+--------+---------+-------+-------------+----------+-------+--------+---------+---------+-----------+------+-------------+-----+--------+----+-----+---+
|S000000124|2023-02-26|     2.0|     60.0| shop_4|San Francisco|California|    USA|   small|    CHA_2|Furniture|      Chair|  null|         48.0| blue|   Basic|2023|    2| 26|
|S000000125|2023-02-25|     1.0|    150.0| shop_5|      Houston|     Texas|    USA|   small|    BED_3|Furniture|        Bed|Single|        127.0|  red|    Mega|2023|    2| 25|
|S000000126|2023-02-24|     1.0|    300.0| shop_6|  San Antonio|     Texas|    USA|   small|    BED_4|Furniture|        

In [ ]:
sales_with_date_components.write.format("delta").partitionBy("month").save("/tmp/delta/deltaSalesDate")

In [ ]:
!ls -R /tmp/delta/deltaSalesDate

/tmp/delta/deltaSalesDate:
 _delta_log  'month=1'	'month=11'  'month=12'	'month=2'

/tmp/delta/deltaSalesDate/_delta_log:
00000000000000000000.json

'/tmp/delta/deltaSalesDate/month=1':
part-00000-2a29956d-cbdc-49b0-af46-d8f1e3112898.c000.snappy.parquet

'/tmp/delta/deltaSalesDate/month=11':
part-00000-9f34f1ba-5ee1-4db7-92a0-af3c4ad5065f.c000.snappy.parquet

'/tmp/delta/deltaSalesDate/month=12':
part-00000-56b99b88-5986-4f01-a020-73dec326588f.c000.snappy.parquet

'/tmp/delta/deltaSalesDate/month=2':
part-00000-2744690d-d0ee-487e-9b86-f2893f5f1a55.c000.snappy.parquet


In [ ]:
sales_with_date_components.write.format("delta").partitionBy("year","month").save("/tmp/delta/deltaSalesDateBis")

In [ ]:
!ls -R /tmp/delta/deltaSalesDateBis

/tmp/delta/deltaSalesDateBis:
 _delta_log  'year=2022'  'year=2023'

/tmp/delta/deltaSalesDateBis/_delta_log:
00000000000000000000.json

'/tmp/delta/deltaSalesDateBis/year=2022':
'month=11'  'month=12'

'/tmp/delta/deltaSalesDateBis/year=2022/month=11':
part-00000-9eff090d-273e-4af6-ac58-90eba94e1a2b.c000.snappy.parquet

'/tmp/delta/deltaSalesDateBis/year=2022/month=12':
part-00000-a5100e1f-1a7f-406a-833c-7297fdd35031.c000.snappy.parquet

'/tmp/delta/deltaSalesDateBis/year=2023':
'month=1'  'month=2'

'/tmp/delta/deltaSalesDateBis/year=2023/month=1':
part-00000-4f7e159f-a790-4f27-be77-af28e51560dc.c000.snappy.parquet

'/tmp/delta/deltaSalesDateBis/year=2023/month=2':
part-00000-d494eb51-a599-49d8-9092-e018fae14ed8.c000.snappy.parquet


#### option2: create a delta table with a predefined schema
https://docs.delta.io/latest/delta-batch.html#use-generated-columns

In [ ]:
DeltaTable.createOrReplace(spark) \
  .tableName("default.sales") \
  .addColumn("saleid", "STRING") \
  .addColumn("saledate", "TIMESTAMP") \
  .addColumn("quantity", "INT") \
  .addColumn("year", "INT", generatedAlwaysAs="YEAR(saledate)") \
  .addColumn("month", "INT", generatedAlwaysAs="MONTH(saledate)") \
  .addColumn("day", "INT", generatedAlwaysAs="DAYOFMONTH(saledate)") \
  .partitionedBy("year", "month") \
  .execute()

In [ ]:
! ls -R spark-warehouse/sales/


spark-warehouse/sales/:
_delta_log

spark-warehouse/sales/_delta_log:
00000000000000000000.json


In [ ]:
spark.sql(""" DESCRIBE DETAIL default.sales """).show()

+------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|                name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|28ec6bf6-48d5-492...|spark_catalog.def...|       null|file:/content/spa...|2023-11-03 03:22:...|2023-11-03 03:22:...|   [year, month]|       0|          0|        {}|               1|               4|[appendOnly, chan...|
+------+--------------------+--------------------+-----------+------------------

In [ ]:
spark.sql(""" select * from default.sales """).show()

+------+--------+--------+----+-----+---+
|saleid|saledate|quantity|year|month|day|
+------+--------+--------+----+-----+---+
+------+--------+--------+----+-----+---+



In [ ]:
spark.sql(""" insert into default.sales
            values ('S000000124','2023-02-26 00:00:00',2.0,2023,02,26)  """).show()

++
||
++
++



In [ ]:
spark.sql(""" select * from default.sales """).show()

+----------+-------------------+--------+----+-----+---+
|    saleid|           saledate|quantity|year|month|day|
+----------+-------------------+--------+----+-----+---+
|S000000124|2023-02-26 00:00:00|       2|2023|    2| 26|
+----------+-------------------+--------+----+-----+---+



if so run a query that aggregates on some measure like sum of `unitprice` based on `month` and observe the plan

In [ ]:
spark.sql(""" DESCRIBE DETAIL delta.`/tmp/delta/deltaSalesDate` """).show()

+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|09332b60-f5b9-4f1...|null|       null|file:/tmp/delta/d...|2023-11-03 03:19:...|2023-11-03 03:19:...|         [month]|       4|      65127|        {}|               1|               2|[appendOnly, inva...|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+---------

In [ ]:
spark.sql(""" select * from delta.`/tmp/delta/deltaSalesDate` """).show(2)

+----------+----------+--------+---------+-------+-------+-------+-------+--------+---------+--------+-----------+----+-------------+-----+-----+----+-----+---+
|    saleid|  saledate|quantity|unitprice| shopid|   city|  state|country|shopsize|productid|category|subcategory|size|purchaseprice|color|brand|year|month|day|
+----------+----------+--------+---------+-------+-------+-------+-------+--------+---------+--------+-----------+----+-------------+-----+-----+----+-----+---+
|S000000212|2022-11-30|     3.0|     20.0|shop_12| Chieti|Abruzzo|  Italy|  medium|   TSH_29|   Cloth|     Tshirt|   S|         18.0|brown| Mega|2022|   11| 30|
|S000000213|2022-11-29|     1.0|     22.0|shop_13|Pescara|Abruzzo|  Italy|   small|   TSH_30|   Cloth|     Tshirt|   M|         19.0|black| Over|2022|   11| 29|
+----------+----------+--------+---------+-------+-------+-------+-------+--------+---------+--------+-----------+----+-------------+-----+-----+----+-----+---+
only showing top 2 rows



In [ ]:
spark.sql(""" select month, sum(unitprice) from delta.`/tmp/delta/deltaSalesDate` group by month """).show()

+-----+--------------+
|month|sum(unitprice)|
+-----+--------------+
|   12|      154648.0|
|    1|      154013.0|
|   11|      136472.0|
|    2|      141531.0|
+-----+--------------+



In [ ]:
spark.conf.set("spark.sql.adaptive.enabled",False)

In [ ]:
spark.sql(""" select month, sum(unitprice) from delta.`/tmp/delta/deltaSalesDate` group by month """).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[month#47749], functions=[sum(unitprice#47735)])
+- Exchange hashpartitioning(month#47749, 200), ENSURE_REQUIREMENTS, [plan_id=18539]
   +- *(1) HashAggregate(keys=[month#47749], functions=[partial_sum(unitprice#47735)])
      +- *(1) ColumnarToRow
         +- FileScan parquet [unitprice#47735,month#47749] Batched: true, DataFilters: [], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[file:/tmp/delta/deltaSalesDate], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<unitprice:double>




Run a different query that aggregates on some measure like sum of `unitprice` based on  `saledate` and compare the plan with the previous one

In [ ]:
spark.sql(""" select year, sum(unitprice) from delta.`/tmp/delta/deltaSalesDate` group by year """).show()

+----+--------------+
|year|sum(unitprice)|
+----+--------------+
|2023|      295544.0|
|2022|      291120.0|
+----+--------------+



In [ ]:
spark.sql(""" select year, sum(unitprice) from delta.`/tmp/delta/deltaSalesDate` group by year """).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[year#47969], functions=[sum(unitprice#47956)])
+- Exchange hashpartitioning(year#47969, 200), ENSURE_REQUIREMENTS, [plan_id=18675]
   +- *(1) HashAggregate(keys=[year#47969], functions=[partial_sum(unitprice#47956)])
      +- *(1) Project [unitprice#47956, year#47969]
         +- *(1) ColumnarToRow
            +- FileScan parquet [unitprice#47956,year#47969,month#47970] Batched: true, DataFilters: [], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[file:/tmp/delta/deltaSalesDate], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<unitprice:double,year:int>




### Retore a delta table to a previous state


In [ ]:
spark.sql("""DESCRIBE HISTORY  default.sales """).show()


+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2023-11-03 03:23:...|  null|    null|               WRITE|{mode -> Append, ...|null|    null|     null|          0|  Serializable|         true|{numFiles -> 1, n...|        null|Apache-Spark/3.4....|
|      0|2023-11-03 03:22:...|  null|    null|CREATE OR REPLACE...|{isManaged -> tru...|null|    null|     null|       null|  Serializable|         true|           

In [ ]:
spark.sql("""SELECT * FROM default.sales VERSION AS OF 1;""").show()


+----------+-------------------+--------+----+-----+---+
|    saleid|           saledate|quantity|year|month|day|
+----------+-------------------+--------+----+-----+---+
|S000000124|2023-02-26 00:00:00|       2|2023|    2| 26|
+----------+-------------------+--------+----+-----+---+



## Use case2


### Table creation

#### sales table
create a delta table `default.sales` with the following schema (saleid : String, saledate : Timestamp, productid: String, quantity : int, shopid : string)

In [ ]:
DeltaTable.createOrReplace(spark) \
  .tableName("default.sales") \
  .addColumn("saleid", "STRING") \
  .addColumn("saledate", "TIMESTAMP") \
  .addColumn("productid", "STRING") \
  .addColumn("quantity", "DOUBLE") \
  .addColumn("shopid", "STRING") \
  .execute()

In [ ]:
schema = spark.sql(""" DESCRIBE default.sales """)
schema.show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|   saleid|   string|   null|
| saledate|timestamp|   null|
|productid|   string|   null|
| quantity|   double|   null|
|   shopid|   string|   null|
+---------+---------+-------+



load the `/tmp/sales/march23_sales.csv` data into `default.sales` by selecting only the required columns

In [ ]:
march23_sales = spark.read.csv("/tmp/delta/sales/march23_sales.csv", header=True, inferSchema=True)

# Select only the required columns
data_cols = march23_sales.select("saleid", "saledate", "productid", "quantity", "shopid")
data_cols = data_cols.withColumn("saledate", col("saledate").cast(TimestampType()))

data_cols.write.format("delta").mode("append").saveAsTable("default.sales")

see the result

In [ ]:
spark.sql(""" select count(*) from default.sales """).show()

+--------+
|count(1)|
+--------+
|      84|
+--------+



#### dates table
create a delta table `default.dates` with the following schema (saledate: timestamp, year: int, month: int) by ensuring that year and month are extracted from saledate

populate `default.dates` by inserting dates from `default.sales`

see the result

In [ ]:
spark.sql(""" select * from default.dates limit 10 """).show()

#### products table
create a delta table default.products with the following schema (productid: string, unitprice: double, category: string, subcategory: string, size: string, color: string,
 brand: string) by extracting data from `originalSales.csv`

see the result

In [ ]:
spark.sql(""" select * from default.products limit 10 """).show()